## NLP on UserReviews - Word2Vec

In [1]:
reviews_df = spark.read.csv("GoogleReviews.csv", header="true", inferSchema="true")

NameError: name 'spark' is not defined

In [0]:
reviews_df.show()

+---+--------------------+--------------------+--------------------+------------------+----------------------+
|_c0|                 App|   Translated_Review|           Sentiment|Sentiment_Polarity|Sentiment_Subjectivity|
+---+--------------------+--------------------+--------------------+------------------+----------------------+
|  0|10 Best Foods for...|"I like eat delic...| also ""Best Befo...|          Positive|                   1.0|
|  1|10 Best Foods for...|This help eating ...|            Positive|              0.25|    0.2884615384615384|
|  2|10 Best Foods for...|                null|                null|              null|                  null|
|  3|10 Best Foods for...|Works great espec...|            Positive|               0.4|                 0.875|
|  4|10 Best Foods for...|        Best idea us|            Positive|               1.0|                   0.3|
|  5|10 Best Foods for...|            Best way|            Positive|               1.0|                   0.3|
|

In [0]:
from pyspark.sql.functions import col
reviews_1 = reviews_df.select('App','Translated_Review','Sentiment').where((col('Sentiment') == 'Positive') | (col('Sentiment') == 'Negative') | (col('Sentiment') == 'Neutral'))
reviews_1.show()

+--------------------+--------------------+---------+
|                 App|   Translated_Review|Sentiment|
+--------------------+--------------------+---------+
|10 Best Foods for...|This help eating ...| Positive|
|10 Best Foods for...|Works great espec...| Positive|
|10 Best Foods for...|        Best idea us| Positive|
|10 Best Foods for...|            Best way| Positive|
|10 Best Foods for...|             Amazing| Positive|
|10 Best Foods for...|Looking forward app,|  Neutral|
|10 Best Foods for...|It helpful site !...|  Neutral|
|10 Best Foods for...|           good you.| Positive|
|10 Best Foods for...|Useful informatio...| Positive|
|10 Best Foods for...|Thank you! Great ...| Positive|
|10 Best Foods for...|Greatest ever Com...| Positive|
|10 Best Foods for...|Good health.........| Positive|
|10 Best Foods for...|Health It's impor...| Positive|
|10 Best Foods for...|Mrs sunita bhati ...| Positive|
|10 Best Foods for...|Very Useful in di...| Positive|
|10 Best Foods for...|  One 

In [0]:
reviews_2 = reviews_1.select('App','Translated_Review','Sentiment').where(col('Translated_Review') != 'null')
reviews_2.show()

+--------------------+--------------------+---------+
|                 App|   Translated_Review|Sentiment|
+--------------------+--------------------+---------+
|10 Best Foods for...|This help eating ...| Positive|
|10 Best Foods for...|Works great espec...| Positive|
|10 Best Foods for...|        Best idea us| Positive|
|10 Best Foods for...|            Best way| Positive|
|10 Best Foods for...|             Amazing| Positive|
|10 Best Foods for...|Looking forward app,|  Neutral|
|10 Best Foods for...|It helpful site !...|  Neutral|
|10 Best Foods for...|           good you.| Positive|
|10 Best Foods for...|Useful informatio...| Positive|
|10 Best Foods for...|Thank you! Great ...| Positive|
|10 Best Foods for...|Greatest ever Com...| Positive|
|10 Best Foods for...|Good health.........| Positive|
|10 Best Foods for...|Health It's impor...| Positive|
|10 Best Foods for...|Mrs sunita bhati ...| Positive|
|10 Best Foods for...|Very Useful in di...| Positive|
|10 Best Foods for...|  One 

In [0]:
reviews_2.count()

Out[8]: 36779

In [0]:
from pyspark.sql.functions import col
data = reviews_2.groupBy('Sentiment').count()
data.show()

+---------+-----+
|Sentiment|count|
+---------+-----+
| Positive|23550|
|  Neutral| 5128|
| Negative| 8101|
+---------+-----+



In [0]:

import plotly.graph_objects as go
df_pd = data.toPandas()

fig = go.Figure(data=[go.Pie(labels=df_pd['Sentiment'], values=df_pd['count'])])
fig.update_layout( title="Pie chart of User sentiments",
                 legend_title="Sentiments")
fig.show()

In [0]:
reviews = reviews_2.toPandas()
reviews

,App,Translated_Review,Sentiment
0,10 Best Foods for You,This help eating healthy exercise regular basis,Positive
1,10 Best Foods for You,Works great especially going grocery store,Positive
2,10 Best Foods for You,Best idea us,Positive
3,10 Best Foods for You,Best way,Positive
4,10 Best Foods for You,Amazing,Positive
...,...,...,...
36774,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive
36775,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive
36776,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative
36777,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive


In [0]:
pip install nltk

Python interpreter will be restarted.
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached regex-2022.4.24-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
Python interpreter will be restarted.


In [0]:
pip install stop_words

Python interpreter will be restarted.
  Using cached stop_words-2018.7.23-py3-none-any.whl
Python interpreter will be restarted.


In [0]:
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk import sent_tokenize, word_tokenize
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

In [0]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [0]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
reviews["review_clean"] = reviews["Translated_Review"].apply(lambda x: clean_text(str(x)))

In [0]:
reviews

,App,Translated_Review,Sentiment,review_clean
0,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,help eat healthy exercise regular basis
1,10 Best Foods for You,Works great especially going grocery store,Positive,work great especially go grocery store
2,10 Best Foods for You,Best idea us,Positive,best idea
3,10 Best Foods for You,Best way,Positive,best way
4,10 Best Foods for You,Amazing,Positive,amaze
...,...,...,...,...
36774,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,ad older many agent much owner post detail rep...
36775,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,photo post portal load fit purpose i'm sure st...
36776,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,dumb app want post property rent give option w...
36777,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,property business get link sm happy performanc...


In [0]:
corpus = []
for col in reviews.review_clean:
   word_list = col.split(" ")
   corpus.append(word_list)

#show first value
corpus[0:1]


Out[42]: [['help', 'eat', 'healthy', 'exercise', 'regular', 'basis']]

In [0]:
#generate vectors from corpus
model = Word2Vec(corpus, min_count=1)

In [0]:
#get similar words from the vocabulary
model.wv.most_similar('photo')



Out[57]: [('pdf', 0.9266800880432129),
 ('file', 0.9132465720176697),
 ('picture', 0.9019389152526855),
 ('gallery', 0.8957309722900391),
 ('document', 0.894316554069519),
 ('folder', 0.8812699317932129),
 ('upload', 0.8773719668388367),
 ('crop', 0.8767473101615906),
 ('print', 0.8763816952705383),
 ('location', 0.8698791861534119)]